In [21]:
# Importing Python packages

import pandas as pd
import numpy as np
import plotnine as p9
import os
import geopandas as gpd
from pathlib import Path


# Data ingest

In [22]:
# Reading in masterplan data

data_masterplan_input = pd.read_csv('mp_data.csv')
data_masterplan_input.head(1)

,Municipality,ProjectID,Include in Summary,Project Name,DistrictID,District,Regional_Subregional Centre_District,District Centre,Centre or Non Centre,Approved,...,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040
0,Abu Dhabi,23037.4,No,Bloom District Phase 2,400,ZAYED CITY,ZAYED CITY,0,NonCentre,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# Reading in TMP 2040 data

tmp_2040_input = pd.read_csv('20240605_TAZ_TMP_2040.csv')
tmp_2040_input.head(1)


,Z,Unnamed: 1,Master plan,PRODSECTOR,AREATYPE,AREATYPE2,UPC_PRECIN,PRECINCT_,HH,POP_NONLAB,...,SECT_INT,REG,TOD_A,TOD_P,HGV_PERMIT,ADSICDIST,DISTNAME,TMP,UDM_DIST,MP_ProjectID
0,1.0,ADM,MP,2.0,9.0,11.0,28.0,Saadiyat Island,283.0,2150.0,...,0.0,1.0,0.0,0.0,1.0,67.0,SADIYAT ISLAND,Saadiyat Lagoons TMP - Saadiyat,AL SAADIYAT ISLAND,1007.42.4


In [24]:

# Reading in land use crosswalk

land_use_crosswalk = pd.read_csv('taxonomy_crosswalks.csv')
land_use_crosswalk = land_use_crosswalk[land_use_crosswalk['CROSSWALK'] == 'MP to TMP 2040 land use']
land_use_crosswalk = land_use_crosswalk[['VARIABLE_1','VARIABLE_2']]
land_use_crosswalk.head(1)

,VARIABLE_1,VARIABLE_2
17,Residential (GFA),RES_GFA


In [52]:
# Reading in TMP summary land use cross wale

tmp_summary_crosswalk = pd.read_csv('taxonomy_crosswalks.csv')
tmp_summary_crosswalk = tmp_summary_crosswalk[tmp_summary_crosswalk['CROSSWALK'] == 'TMP 2040 to Summary 2040 land uses']
tmp_summary_crosswalk = tmp_summary_crosswalk[['VARIABLE_1','VARIABLE_2']]
tmp_summary_crosswalk.head(3)

,VARIABLE_1,VARIABLE_2
27,RES_GFA,RES_GFA
28,RETAIL_GFA,RETAIL_GFA
29,OFFICE_GFA,OFFICE_GFA


In [26]:
# Reading in unit size assumptions

unit_densities = pd.read_csv('land_use_gfa_densities.csv')
unit_densities

,land_use,size_gfa,source
0,Residential - Villas Emiratis,700,CBRE estimate
1,Residential - Villas Non-Emiratis,400,CBRE estimate
2,Residential - Apartments (Units),125,CBRE estimate
3,Hotel (Units),80,CBRE estimate
4,Labour Accommodation (Bed),10,CBRE estimate


# Converting Master Plan Development to TMP Summary Land Uses

In [27]:
# Setting gfa per labourer bed density

labour_bed_gfa = unit_densities.iloc[4]['size_gfa'].tolist()

# Creating labour accommodation size column in the main masterplan input worksheet

data_masterplan = data_masterplan_input.copy()
data_masterplan['Residential - Labour Accommodation (Units)'] = pd.to_numeric(data_masterplan['Residential - Labour Accommodation (Units)'],errors= 'coerce')
data_masterplan['Residential Labour (GFA)'] = data_masterplan['Residential - Labour Accommodation (Units)'] * labour_bed_gfa
data_masterplan.head(1)

,Municipality,ProjectID,Include in Summary,Project Name,DistrictID,District,Regional_Subregional Centre_District,District Centre,Centre or Non Centre,Approved,...,2032,2033,2034,2035,2036,2037,2038,2039,2040,Residential Labour (GFA)
0,Abu Dhabi,23037.4,No,Bloom District Phase 2,400,ZAYED CITY,ZAYED CITY,0,NonCentre,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [28]:
# Setting hotel room size

hotel_room_gfa = unit_densities.iloc[3]['size_gfa']

# Convert to numeric, forcing errors to NaN
data_masterplan['Hotel (Units)'] = pd.to_numeric(data_masterplan['Hotel (Units)'], errors='coerce')
data_masterplan['Hotel (Units)'] = data_masterplan['Hotel (Units)'].round(decimals=0).astype('Int64')# Round the values and convert to integers
data_masterplan['Hotel (GFA)'] = data_masterplan.apply(
    lambda row: row['Hotel (Units)'] * hotel_room_gfa if pd.notna(row['Hotel (Units)']) and row['Hotel (Units)'] > 0 and row['Hotel (GFA)'] == 0 else row['Hotel (GFA)'],
    axis=1)# Apply the function to calculate 'Hotel (GFA)'
data_masterplan.head(1)

,Municipality,ProjectID,Include in Summary,Project Name,DistrictID,District,Regional_Subregional Centre_District,District Centre,Centre or Non Centre,Approved,...,2032,2033,2034,2035,2036,2037,2038,2039,2040,Residential Labour (GFA)
0,Abu Dhabi,23037.4,No,Bloom District Phase 2,400,ZAYED CITY,ZAYED CITY,0,NonCentre,Yes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [29]:
# Creating lists
mp_land_uses = land_use_crosswalk['VARIABLE_1'].drop_duplicates().tolist()

In [78]:
# Turning masterplan GFAs into long form

mp_space_long = data_masterplan[mp_land_uses]
mp_space_long = pd.concat([data_masterplan[['ProjectID']],mp_space_long],axis = 1)
mp_space_long = mp_space_long.melt(id_vars= ['ProjectID']).rename({'variable':'VARIABLE_1'},axis =1)
mp_space_long['value'] = pd.to_numeric(mp_space_long['value'])
mp_space_long['value'] = mp_space_long['value'].fillna(0).reset_index(drop = True)
mp_space_long.head(1)

,ProjectID,VARIABLE_1,value
0,23037.4,Residential (GFA),0.0


In [31]:
# Converting masterplan GFAs over to TMP land uses

mp_to_tmp_space = mp_space_long.merge(land_use_crosswalk,how = 'left', on = 'VARIABLE_1').dropna()
mp_to_tmp_space['value'] = pd.to_numeric(mp_to_tmp_space['value'],errors = 'coerce')
mp_to_tmp_space = mp_to_tmp_space.groupby(['ProjectID','VARIABLE_2'])['value'].agg('sum').reset_index()
mp_to_tmp_space.head(1)

,ProjectID,VARIABLE_2,value
0,10005,HOTELROOMS,0.0


# Transferring TMP 2040 over to summary TMP land uses

In [32]:
# Creating list containing tmp land uses

tmp_summary_uses = tmp_summary_crosswalk['VARIABLE_1'].drop_duplicates().tolist()
print(tmp_summary_uses)

['RES_GFA', 'RETAIL_GFA', 'OFFICE_GFA', 'L_IND_GFA', 'M_IND_GFA', 'H_IND_GFA', 'SCHOOL_GFA', 'MED_GFA', 'HOTELROOMS', 'OTHER_GFA']


In [49]:
# Version of tmp_2040_input

tmp_2040_input_no_na  = tmp_2040_input.copy()
tmp_2040_input_no_na = tmp_2040_input_no_na[~tmp_2040_input_no_na['Z'].isna()]
tmp_2040_input_no_na.head(1)

,Z,Unnamed: 1,Master plan,PRODSECTOR,AREATYPE,AREATYPE2,UPC_PRECIN,PRECINCT_,HH,POP_NONLAB,...,SECT_INT,REG,TOD_A,TOD_P,HGV_PERMIT,ADSICDIST,DISTNAME,TMP,UDM_DIST,MP_ProjectID
0,1.0,ADM,MP,2.0,9.0,11.0,28.0,Saadiyat Island,283.0,2150.0,...,0.0,1.0,0.0,0.0,1.0,67.0,SADIYAT ISLAND,Saadiyat Lagoons TMP - Saadiyat,AL SAADIYAT ISLAND,1007.42.4


In [76]:

# Creating summary dataframe with masterplan IDs and relevant development level columns

tmp_space = tmp_2040_input[tmp_summary_uses].dropna() # Dropping rows containing nas
tmp_space = pd.concat([tmp_2040_input_no_na[['Z']],tmp_space],axis = 1) # Concatenating with dataframe that has TAZ labels
tmp_space = tmp_space.melt(id_vars= 'Z').rename({'variable':'VARIABLE_1'},axis = 1) # Melting into long form
tmp_space = tmp_space.merge(tmp_summary_crosswalk,how = 'left', on = 'VARIABLE_1') # Merging on summary land uses to long form dataframe
tmp_space = tmp_space.groupby(['Z','VARIABLE_2'])['value'].agg('sum').reset_index() # Summing floorspace (i.e. value column) using groupby TAZ and summary land use logic
tmp_space['per'] = tmp_space.groupby('Z')['value'].transform(lambda x: x / x.sum()) # Creating column that contains the percentage of floorspace by land use in each TAZ
tmp_space.head(5)

,Z,VARIABLE_2,value,per
0,1.0,HOTELROOMS,0.0,0.000000
1,1.0,IND_GFA,0.0,0.000000
2,1.0,MED_GFA,0.0,0.000000
3,1.0,OFFICE_GFA,0.0,0.000000
4,1.0,OTHER_GFA,1900.0,0.012034


In [77]:
tmp_space

,Z,VARIABLE_2,value,per
0,1.0,HOTELROOMS,0.0,0.000000
1,1.0,IND_GFA,0.0,0.000000
2,1.0,MED_GFA,0.0,0.000000
3,1.0,OFFICE_GFA,0.0,0.000000
4,1.0,OTHER_GFA,1900.0,0.012034
...,...,...,...,...
27803,3476.0,OFFICE_GFA,24870.0,0.007300
27804,3476.0,OTHER_GFA,5200.0,0.001526
27805,3476.0,RES_GFA,0.0,0.000000
27806,3476.0,RETAIL_GFA,3100.0,0.000910


In [53]:
tmp_2040_input

,Z,Unnamed: 1,Master plan,PRODSECTOR,AREATYPE,AREATYPE2,UPC_PRECIN,PRECINCT_,HH,POP_NONLAB,...,SECT_INT,REG,TOD_A,TOD_P,HGV_PERMIT,ADSICDIST,DISTNAME,TMP,UDM_DIST,MP_ProjectID
0,1.0,ADM,MP,2.0,9.0,11.0,28.0,Saadiyat Island,283.0,2150.0,...,0.0,1.0,0.0,0.0,1.0,67.0,SADIYAT ISLAND,Saadiyat Lagoons TMP - Saadiyat,AL SAADIYAT ISLAND,1007.42.4
1,2.0,ADM,MP,2.0,9.0,11.0,28.0,Saadiyat Island,168.0,1283.0,...,0.0,1.0,0.0,0.0,1.0,67.0,SADIYAT ISLAND,Saadiyat Lagoons TMP - Saadiyat,AL SAADIYAT ISLAND,1007.42.4
2,3.0,ADM,MP,2.0,3.0,11.0,28.0,Saadiyat Island,19.0,47.0,...,0.0,1.0,0.0,0.0,1.0,67.0,SADIYAT ISLAND,STEAM_v132_Cultural_District,AL SAADIYAT ISLAND,1007.0
3,4.0,ADM,MP,2.0,1.0,11.0,28.0,Saadiyat Island,411.0,1255.0,...,0.0,1.0,0.0,0.0,1.0,67.0,SADIYAT ISLAND,STEAM_v132_ NYU_Precinct,AL SAADIYAT ISLAND,1007.27
4,5.0,ADM,MP,2.0,9.0,11.0,28.0,Saadiyat Island,146.0,1115.0,...,0.0,1.0,0.0,0.0,1.0,67.0,SADIYAT ISLAND,Saadiyat Lagoons TMP - Saadiyat,AL SAADIYAT ISLAND,1007.42.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# Reading in baseline TAZ data

sharepoint_path = f'C:/Users/{os.environ.get( "USERNAME" )}/Buro Happold'
project_path = Path(f"{sharepoint_path}/P060927 Plan Capital 2040 - 02_Documents/11_GIS/05 Urban Models")
ium_path = os.path.join(project_path,'ium_ad.gpkg')
zone_taz_2022 = gpd.read_file(ium_path, layer='zone_taz_2022')
zone_taz_2022.head(1)


,Z,SHAPE_Length,SHAPE_Area,NUM_OF_BUILDINGS,Calc_Total_Ex_GFA,Ex_GFA,RES_GFA,Residential_Labour_GFA,OFFICE_GFA,RETAIL_GFA,...,RETMAL_PRO,MEDCLI_PRO,MEDHOS_PRO,NONTER_PRO,TER_PRO,SCHPLCS_NT,SCHPLCS_TE,SCHPLCS_PR,SCHPLCS_SE,geometry
0,1044,47079.0,606869.0,679.0,297016.0,297542.0,271410.0,0.0,0.0,0.0,...,None,None,None,None,None,None,None,None,None,"MULTIPOLYGON (((270352.336 2727199.707, 269735..."


In [35]:
# Reading in taz zone delta

zone_taz_delta = gpd.read_file(ium_path, layer='zone_taz_delta')
zone_taz_delta.head(1)


,Z,ZONE,EPA,UPC_PRECIN,PRECINCT_N,ST_DIST_ID,ST_DIST_EN,UDM_DIST_I,UDM_DIST_E,REGION,...,score_component_rank_sum_rank_mp,score_component_rank_mean_rank_mp,score_component_rank_sum_rank_urban,score_component_rank_mean_rank_urban,score_development_priority,2022_taz_far,gfa_per_ha,landmodel_limit_gfa_per_ha,landmodel_limit_gfa_per_ha_delta,geometry
0,2510,2510,None,1.0,Abu Dhabi,1.0,Abu Dhabi West Coast And Islands And Mussaffah...,500.0,ABU AL ABYAD ISLAND,ADM,...,NaN,NaN,1.0,1.0,5.0,0.000003,0.026657,0.026657,-0.025877,"MULTIPOLYGON (((182767.838 2687320.951, 183001..."


In [36]:
# Reading in land-use split
land_use_crosswalk


,VARIABLE_1,VARIABLE_2
17,Residential (GFA),RES_GFA
18,Residential Labour (GFA),RES_GFA
19,Office (GFA),OFFICE_GFA
20,Retail (GFA),RETAIL_GFA
21,Hotel (GFA),HOTELROOMS
22,Industrial (GFA),IND_GFA
23,Education (GFA),SCHOOL_GFA
24,Healthcare (GFA),MED_GFA
25,Community Facilities (GFA),OTHER_GFA
26,Other (GFA),OTHER_GFA


In [37]:
tmp_2040_select_columns = ['Z','MP_ProjectID','RES_GFA','RETAIL_GFA','OFFICE_GFA','L_IND_GFA','M_IND_GFA','H_IND_GFA','SCHOOL_GFA','MED_GFA','OTHER_GFA']

In [38]:
tmp_2040 = tmp_2040_input[tmp_2040_select_columns]
tmp_2040 = tmp_2040[~tmp_2040['Z'].isna()]
tmp_2040['Z'] = tmp_2040['Z'].astype(str)
tmp_2040['TOTAL_GFA'] = tmp_2040.select_dtypes(include=[float, int]).sum(axis=1)
tmp_2040.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3476 entries, 0 to 3475
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Z             3476 non-null   object 
 1   MP_ProjectID  3476 non-null   object 
 2   RES_GFA       3476 non-null   float64
 3   RETAIL_GFA    3476 non-null   float64
 4   OFFICE_GFA    3476 non-null   float64
 5   L_IND_GFA     3476 non-null   float64
 6   M_IND_GFA     3476 non-null   float64
 7   H_IND_GFA     3476 non-null   float64
 8   SCHOOL_GFA    3476 non-null   float64
 9   MED_GFA       3476 non-null   float64
 10  OTHER_GFA     3476 non-null   float64
 11  TOTAL_GFA     3476 non-null   float64
dtypes: float64(10), object(2)
memory usage: 353.0+ KB


In [39]:
tmp_2040_per = tmp_2040.select_dtypes(include=[float, int]).div(tmp_2040['TOTAL_GFA'], axis=0)
tmp_2040_per = pd.concat([tmp_2040[['Z','MP_ProjectID']],tmp_2040_per],axis = 1)
tmp_2040_per = tmp_2040_per[tmp_2040['MP_ProjectID'] != '0'].reset_index(drop = True)
tmp_2040_per = tmp_2040_per[~tmp_2040['MP_ProjectID'].isna()].reset_index(drop = True)
tmp_2040_per

C:\Users\jtindale\AppData\Local\Temp\ipykernel_10436\1306470590.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,Z,MP_ProjectID,RES_GFA,RETAIL_GFA,OFFICE_GFA,L_IND_GFA,M_IND_GFA,H_IND_GFA,SCHOOL_GFA,MED_GFA,OTHER_GFA,TOTAL_GFA
0,1.0,1007.42.4,0.986446,0.001520,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012034,1.0
1,2.0,1007.42.4,0.899572,0.007761,0.000000,0.000000,0.000000,0.000000,0.075361,0.000000,0.017306,1.0
2,3.0,1007.0,0.439153,0.000000,0.560847,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
3,4.0,1007.27,0.700105,0.022941,0.000000,0.000000,0.000000,0.000000,0.162524,0.000000,0.114430,1.0
4,5.0,1007.42.4,0.990072,0.000953,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008975,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1649,3472.0,1997.0,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
1650,3473.0,1997.0,0.000000,0.873139,0.076865,0.000000,0.000000,0.000000,0.000000,0.000000,0.049995,1.0
1651,3474.0,1995.0,0.000000,0.006808,0.031058,0.207263,0.377436,0.377436,0.000000,0.000000,0.000000,1.0
1652,3475.0,1995.0,0.000000,0.002353,0.000000,0.199269,0.398537,0.398537,0.000000,0.000858,0.000445,1.0


In [40]:
tmp_2040_input.columns

Index(['Z', 'Unnamed: 1', 'Master plan', 'PRODSECTOR', 'AREATYPE', 'AREATYPE2',
       'UPC_PRECIN', 'PRECINCT_', 'HH', 'POP_NONLAB', 'WORKER', 'STUDENT',
       'LABOURER', 'POP_TOT', 'HOTELROOMS', 'RES_GFA', 'RETAIL_GFA',
       'OFFICE_GFA', 'L_IND_GFA', 'M_IND_GFA', 'H_IND_GFA', 'SCHOOL_GFA',
       'MED_GFA', 'OTHER_GFA', 'CONS_SITE', 'RETGEN_PRO', 'RETMAL_PRO',
       'MEDCLI_PRO', 'MEDHOS_PRO', 'NONTER_PRO', 'TER_PRO', 'SCHPLCS_NT',
       'SCHPLCS_TE', 'SCHPLCS_PR', 'SCHPLCS_SE', 'SECT_DST', 'SECT_FRT',
       'SECT_EXT', 'SECT_REP', 'SECT_REPMS', 'METRO_AREA', 'SHAPEAREA',
       'CENTROIDX', 'CENTROIDY', 'PARK_SEC', 'URBAN_RURA', 'LAB_CON_P',
       'LAB_OTH_P', 'LAB_IND_P', 'IND_AREA', 'PROP_INT_L', 'PROP_INT_W',
       'PROP_INT_O', 'SECT_BAL', 'SECT_INT', 'REG', 'TOD_A', 'TOD_P',
       'HGV_PERMIT', 'ADSICDIST', 'DISTNAME', 'TMP', 'UDM_DIST',
       'MP_ProjectID'],
      dtype='object')